![MGB Logo](https://github.com/v-mourajr/mongan_llm_workshop/blob/master/images/mgb-logo.png?raw=true)

# Using LLM for Clinical Note Analysis
This Notebook demonstrates how to use Azure-hosted OpenAI models with LangChain. The notebook follows a step-by-step approach to set up authentication, initialize the model, create structured prompts, and generate responses. The key components covered include:#

- Loading environment variables
- Setting up Azure authentication
- Creating and formatting chat prompts
- Invoking the OpenAI model
- Displaying the AI-generated response

Each cell in the notebook builds upon the previous one to progressively set up and execute an AI-driven conversation.

In [98]:
# -----------------------------------------------------------
# 1. Importing Required Libraries
# -----------------------------------------------------------
# Demonstrates the necessary imports for setting up Azure OpenAI authentication and model interaction.

# Key Components:
#   - os: Provides functions for interacting with the operating system, such as accessing environment variables.
#   - load_dotenv(): Loads environment variables from a .env file to securely store sensitive credentials.
#   - DefaultAzureCredential: Handles authentication with Azure services using various credential methods.
#   - get_bearer_token_provider(): Retrieves an authentication token for accessing Azure OpenAI services.
#   - AzureChatOpenAI: A LangChain wrapper for interacting with Azure-hosted OpenAI models.

# Purpose:
# These libraries enable secure authentication and seamless integration of OpenAI models within Azure,
# ensuring that AI capabilities can be efficiently accessed and utilized in applications.

import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureChatOpenAI

print("\n=== Required Libraries Loaded ===")


In [109]:
# -----------------------------------------------------------
# 2. Setting Up Azure OpenAI Model
# -----------------------------------------------------------
# Demonstrates how to configure and authenticate Azure OpenAI for use in AI-driven applications.

# Key Components:
#   - load_dotenv(): Loads environment variables from a .env file to securely store credentials.
#   - DefaultAzureCredential(): Uses Azure’s default authentication method to retrieve credentials.
#   - get_bearer_token_provider(): Retrieves an authentication token for accessing Azure OpenAI services.
#   - AzureChatOpenAI(): Initializes the Azure-hosted OpenAI model using environment variables.
#     - openai_api_version: Specifies the API version to use.
#     - azure_deployment: Identifies the specific OpenAI deployment.
#     - azure_endpoint: Defines the Azure endpoint for API access.
#     - azure_ad_token_provider: Supplies the necessary authentication token.

# Purpose:
# This setup ensures secure and efficient access to Azure OpenAI services, enabling seamless integration of AI capabilities into applications.
# It provides a scalable way to interact with OpenAI models hosted on Azure while managing authentication automatically.

load_dotenv()

# Set up Azure credentials and token provider
azure_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"
)

# Initialize the AzureChatOpenAI model using environment variables
model = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider
)

print("\n=== LLM Models Loaded ===")



=== LLM Models Loaded ===


## 3. Basic Prompt Interaction
<img src="https://github.com/v-mourajr/mongan_llm_workshop/blob/master/images/basic_prompt.png?raw=true" alt="Basic Prompt" width="800">

In [100]:
# -----------------------------------------------------------
# 3. Basic Prompt Interaction
# -----------------------------------------------------------
# Demonstrates a simple prompt-based interaction with an AI model using LangChain.
 
# Key Components:
#   - SystemMessage: Defines the AI’s role as a medical provider, ensuring responses are evidence-based and patient-friendly.
#   - HumanMessage: Represents the user's query, in this case: "What is asthma? What are its common symptoms and treatments?"
#   - model.invoke(messages): Sends the conversation history to the AI model, prompting it to generate a response.

# Purpose: 
# This setup enables structured AI interactions, ensuring the model responds consistently and accurately in a medical context.
# Can be extended to handle more complex medical queries or chatbot interactions.

from langchain_core.messages import HumanMessage, SystemMessage

# Define a chat interaction using a system message (to set the AI’s role) and a human message (a user query about asthma).

messages = [
    SystemMessage(content="You are a knowledgeable medical provider with expertise in diagnosing and managing various diseases. Provide clear, evidence-based, and patient-friendly explanations about medical conditions, symptoms, and treatments."),
    
    HumanMessage(content="What is asthma? What are its common symptoms and treatments?")
]

# Invoke the model with messages
result = model.invoke(messages)

print(result.content)

Asthma is a chronic respiratory condition characterized by inflammation and narrowing of the airways, which can make breathing difficult. This condition can vary in severity and may trigger during specific situations, such as exercise or exposure to allergens.

### Common Symptoms of Asthma
1. **Shortness of Breath:** Difficulty breathing or feeling breathless.
2. **Wheezing:** A squeaky or whistling sound when you breathe.
3. **Chest Tightness:** A feeling of pressure or tightness in the chest.
4. **Coughing:** Particularly at night or early morning, or when exposed to cold air or exercise.
5. **Difficulty Sleeping:** Due to shortness of breath, coughing, or wheezing.

### Triggers
Certain factors can exacerbate asthma symptoms, including:
- Allergens (pollen, dust mites, pet dander)
- Irritants (smoke, pollution, strong odors)
- Physical activity
- Cold air
- Respiratory infections (such as colds or the flu)
- Stress and strong emotions

### Treatment Options
1. **Long-term control m

In [110]:
# -----------------------------------------------------------
# 4. Using ChatPromptTemplate for Dynamic Queries
# -----------------------------------------------------------
# Demonstrates how to create a reusable prompt template with dynamic input variables using LangChain.

# Key Components:
#   - ChatPromptTemplate: Allows dynamic variables in prompts, making interactions more flexible.
#   - SystemMessage: Sets the AI’s role as a medical provider, ensuring responses are clear and evidence-based.
#   - HumanMessage: Contains a query template with a placeholder ({disease}), allowing different medical conditions to be queried dynamically.
#   - prompt_template.invoke({"disease": disease}): Fills in the variable "disease" with the specified condition (e.g., "epilepsy").
#   - model.invoke(prompt): Sends the formatted query to the AI model for a response.

# Purpose:
# This approach enables reusable prompts where users can query different diseases without modifying the prompt structure.
# It improves scalability and efficiency for medical applications or chatbots handling multiple medical conditions.


from langchain.prompts import ChatPromptTemplate

ai_role = "Neurologist"
disease = "epilepsy"

messages = [
    ("system", f"You are a knowledgeable {ai_role} with expertise in diagnosing and managing various diseases. Provide clear, evidence-based, and patient-friendly explanations about medical conditions, symptoms, and treatments."),
    
    ("human", "What is {disease}? What are its common symptoms and treatments?"),
]
# Create PromptTemplate
prompt_template = ChatPromptTemplate.from_messages(messages)

# Plug-in user variables
prompt = prompt_template.invoke({"ai_role": ai_role , "disease": disease})

# invoke model
result = model.invoke(prompt)
print(result.content)

PermissionDeniedError: Error code: 403 - {'error': {'code': '403', 'message': 'Public access is disabled. Please configure private endpoint.'}}

In [102]:
# -----------------------------------------------------------
# 5. Using ChatPromptTemplate for Clinical Note Analysis
# -----------------------------------------------------------
# Demonstrates how to create a structured prompt template for extracting medical details from clinical notes.

# Key Components:
#   - ChatPromptTemplate: Creates a reusable prompt structure for analyzing clinical notes.
#   - SystemMessage: Defines the AI’s role as a medical documentation assistant, ensuring accurate extraction of medical details.
#   - HumanMessage: Contains a structured query with a placeholder ({patient_note}), allowing different clinical notes to be analyzed dynamically.
#   - prompt_template_notes.invoke({"patient_note": note_text}): Fills in the variable "patient_note" with the actual clinical note text.
#   - model.invoke(prompt): Sends the formatted query to the AI model for processing.

# Purpose:
# This approach enables automated medical text processing, ensuring structured extraction of relevant information such as demographics, chief complaints, medications, and asthma status.
# It improves efficiency in clinical documentation and can be integrated into medical record systems for automated analysis.


messages_notes = [
    (
        "system", 
        "You are an advanced medical documentation assistant with expertise in clinical text analysis. Your task is to review a given clinical note and extract relevant medical details accurately."
    ),
    
    (
        "human", 
        "Please analyze the following clinical note: {patient_note}. \n\n"
        "Extract and list the following information:\n"
        "1. Patient demographics\n"
        "2. Chief Complaints\n"
        "3. Current Medications\n"
        "4. Determine whether the patient has asthma (Yes/No), based on explicit mentions or related diagnoses.\n\n"
        "Provide the output in a structured format."
    )
]

prompt_template_notes = ChatPromptTemplate.from_messages(messages_notes)


In [107]:
# -----------------------------------------------------------
# 6. Loading and Reading a Patient Note
# -----------------------------------------------------------
# Demonstrates how to load a clinical note from a file for further processing.

# Key Components:
#   - input_dir: Specifies the directory where patient notes are stored.
#   - filename: Defines the specific file to be loaded, containing a patient's medical note.
#   - os.path.join(input_dir, filename): Constructs the full file path dynamically.
#   - open(file_path, "r"): Opens the file in read mode and loads the content into the variable "patient_note".
#   - print(f"\nPatient Note:\n\n{patient_note}"): Displays the loaded note for verification.

# Purpose:
# This step ensures that patient notes are correctly loaded before being analyzed by the AI model.
# It enables seamless integration with document processing pipelines for clinical text analysis.


input_dir = 'data_prep/patient_notes'
filename = 'note_1000000013_20091210.txt' 

# examples without Asthma: 
# -----------------------------------------------------------
# 1000000002, 1000000003, 1000000009, 1000000010, 1000000013, 
# 1000000023, 1000000036, 1000000040, 1000000047, 1000000048, 
# 1000000052, 1000000063, 1000000064, 1000000068, 1000000071, 
# 1000000082, 1000000086, 1000000087, 1000000093, 1000000101, 
# 1000000103, 1000000107
# -----------------------------------------------------------


# Load the note txt
file_path = os.path.join(input_dir, filename)
with open(file_path, "r") as file:
    patient_note = file.read()
    
print(f"\nPatient Note:\n\n{patient_note}")


Patient Note:

This is an 84-year-old Black female who has been receiving care at our clinic for several years and primarily communicates in German with the assistance of a translator. On 12/10/2009, she presents for a routine follow-up. The patient reports feeling generally more fatigued and weaker than usual over the past month. She also describes intermittent dizziness and persistent shortness of breath on exertion. She denies experiencing any fever, night sweats, or unexplained weight loss. 

Her past medical history includes hypertension, chronic kidney disease, congestive heart failure, type II diabetes mellitus, a calcaneal spur, osteoporosis, and has undergone a complete thyroidectomy. Her surgical history also includes endoscopic biopsy of the large intestine and a closed [endoscopic] biopsy for breast screening. Her current medications include Lisinopril for hypertension, Furosemide for congestive heart failure and fluid management, Metformin for diabetes mellitus, and a dai

In [108]:
# -----------------------------------------------------------
# 7. Invoking the AI Model for Clinical Note Analysis
# -----------------------------------------------------------
# Demonstrates how to process a patient note using a structured prompt and invoke the AI model for analysis.

# Key Components:
#   - prompt_template_notes.invoke({"patient_note": patient_note}): 
#     Dynamically fills the prompt template with the actual clinical note text.
#   - model.invoke(prompt_notes): 
#     Sends the formatted query to the AI model for processing.
#   - print(result_note.content): 
#     Displays the AI-generated structured output containing extracted medical details.

# Purpose:
# This approach enables automated extraction of structured medical information from clinical notes.
# It ensures efficient and standardized processing of patient data, aiding in medical documentation and decision-making.

prompt_notes = prompt_template_notes.invoke({"patient_note": patient_note})

# invoke model
result_note = model.invoke(prompt_notes)
print(result_note.content)


1. **Patient demographics:**
   - Age: 84 years old
   - Gender: Female
   - Ethnicity/Race: Black
   - Language: Primarily communicates in German with the assistance of a translator

2. **Chief Complaints:**
   - General fatigue and weakness over the past month
   - Intermittent dizziness
   - Persistent shortness of breath on exertion
   - No fever, night sweats, or unexplained weight loss

3. **Current Medications:**
   - Lisinopril (for hypertension)
   - Furosemide (for congestive heart failure and fluid management)
   - Metformin (for type II diabetes mellitus)
   - Daily aspirin (for cardiovascular prophylaxis)

4. **Asthma Diagnosis:**
   - No (The patient does not have asthma, and there are no explicit mentions of asthma or related diagnoses in the clinical note.)
